In [ ]:
import sys
import os

parent_dir = os.path.dirname(os.getcwd())
folder_path = os.path.join(parent_dir, "src")
sys.path.append(folder_path)

import torch
import numpy as np
import os
import XGBoost_utils
import joblib
import xgboost as xgb
import time
from Magazine_Optimization import *

### Within Magazine

In [ ]:
root_dir = os.path.join(parent_dir, "In-Silico_Experiments_random/Pseudo-Magazines")

#Within Magazines
optimal = []
original = []
random = []

optimal_bg = []
original_bg = []
random_bg = []

optimal_bs = []
original_bs = []
random_bs = []

ags = []

avg_indices = None 
avg_high_level = None 
zero_indices = None 

#To run Observed-features-only experiment, uncomment the following lines:
# avg_indices = np.array(list(range(4,56))+list(range(67,71))+[109])
# avg_high_level = torch.load(os.path.join(parent_dir, 'In-Silico_Experiments/average_nonobserv.pt')) 
# zero_indices = np.array([110])

#To run Low-Level-features-only experiment, uncomment the following lines:
# avg_indices = np.array([20,21,22,23,50,51,52,53,54,55,109])
# avg_high_level = torch.load(os.path.join(parent_dir, 'In-Silico_Experiments/average_high_level.pt'))
# zero_indices = np.array(list(range(56,65))+list(range(67,109))+[110])

for f in range(2):
    print('Magazine '+str(f+1))
    dir = root_dir + '/M' + str(f+1) + '/Var_inputs.pt'
    if os.path.exists(dir):
        Var_inputs = torch.load(dir)
        ag = 0
        bg = 0
        bs = 0
        ag_avg = 0
        bg_avg = 0
        bs_avg = 0

        for k in range(10):
            if avg_indices is not None:
                Var_avg_k = Var_inputs[k].copy()
                Var_avg_k[:, avg_indices] = avg_high_level
                Var_avg_k[:, zero_indices] = 0

            xgb_model = xgb.XGBRegressor()
            xgb_model.load_model(os.path.join(parent_dir,'src/Ad_Gaze_Model/10_models/Model_'+str(k+1)+'.json'))
            
            ag += xgb_model.predict(Var_inputs[k])/10
            if avg_indices is not None:
                ag_avg += xgb_model.predict(Var_avg_k)/10

            xgb_model.load_model(os.path.join(parent_dir,'src/Brand_Gaze_Model/10_models/Model_'+str(k+1)+'.json'))
            bg += xgb_model.predict(Var_inputs[k])/10
            if avg_indices is not None:
                bg_avg += xgb_model.predict(Var_avg_k)/10

            xgb_model.load_model(os.path.join(parent_dir,'src/Brand_Share_Model/10_models/Model_'+str(k+1)+'.json'))
            bs += xgb_model.predict(Var_inputs[k])/10
            if avg_indices is not None:
                bs_avg += xgb_model.predict(Var_avg_k)/10

        ag = np.exp(ag)-1
        bg = np.exp(bg)-1

        if avg_indices is not None:
            ag_avg = np.exp(ag_avg)-1 
            bg_avg = np.exp(bg_avg)-1
            bs_avg = np.exp(bs_avg)-1
        else:
            ag_avg = np.zeros_like(ag) 
            bg_avg = np.zeros_like(bg)
            bs_avg = np.zeros_like(bs)

        N = int(np.sqrt(len(ag)))
        ag = ag.reshape((N,N))
        bg = bg.reshape((N,N))
        bs = bs.reshape((N,N))
        ag_avg = ag_avg.reshape((N,N))
        bg_avg = bg_avg.reshape((N,N))
        bs_avg = bs_avg.reshape((N,N))

        arr = np.arange(N)
        np.random.shuffle(arr)
        for i in range(N):
            random.append(ag[i,arr[i]])
            random_bg.append(bg[i,arr[i]])
            random_bs.append(bs[i,arr[i]])

        workers = []
        jobs = []
        Assign_ids = []
        for i in range(N):
            workers.append(i+1)
            jobs.append(i+1)
            Assign_ids.append(i)

        if avg_indices is None:
            ag_assign = ag
            bg_assign = bg
            bs_assign = bs
        else:
            ag_assign = ag_avg
            bg_assign = bg_avg
            bs_assign = bs_avg

        max_ad_attention = np.max(ag_assign)
        max_brand_attention = np.max(bg_assign)
        max_brand_share = np.max(bs_assign)
        Ad_Gaze_cost = max_ad_attention - ag_assign
        Brand_Gaze_cost = max_brand_attention - bg_assign
        Brand_Share_cost = max_brand_share - bs_assign

        Prob_solved_Ad = Assignment_Problem(Ad_Gaze_cost, workers, jobs)

        Ad_Gaze_true = ag
        Brand_Gaze_true = bg
        Brand_Share_true = bs

        strategy_AG = ''
        BG_under_AG_assignment = 0
        BS_under_AG_assignment = 0
        AG_fine_detail = []
        BG_under_AG_fine_detail = []
        BS_under_AG_fine_detail = []
        AG_fine_detail_true = []
        BG_under_AG_fine_detail_true = []
        BS_under_AG_fine_detail_true = []
        for v in Prob_solved_Ad.variables():
            if v.varValue == 1:
                curr = (v.name).split('_')
                temp = curr[0]+' Ad '+str(Assign_ids[int(curr[1])-1])+' to Counterpage '+str(Assign_ids[int(curr[2])-1])
                AG_fine_detail.append(max_ad_attention-Ad_Gaze_cost[int(curr[1])-1,int(curr[2])-1])
                AG_fine_detail_true.append(Ad_Gaze_true[int(curr[1])-1,int(curr[2])-1])

                BG_under_AG_assignment += Brand_Gaze_cost[int(curr[1])-1,int(curr[2])-1]
                BG_under_AG_fine_detail.append(max_brand_attention-Brand_Gaze_cost[int(curr[1])-1,int(curr[2])-1])
                BG_under_AG_fine_detail_true.append(Brand_Gaze_true[int(curr[1])-1,int(curr[2])-1])

                BS_under_AG_assignment += Brand_Share_cost[int(curr[1])-1,int(curr[2])-1]
                BS_under_AG_fine_detail.append(max_brand_share-Brand_Share_cost[int(curr[1])-1,int(curr[2])-1])
                BS_under_AG_fine_detail_true.append(Brand_Share_true[int(curr[1])-1,int(curr[2])-1])
                strategy_AG += temp+'; '
                # print(temp)
        
        optimal += AG_fine_detail_true
        original += list(np.diag(ag))
        optimal_bg += BG_under_AG_fine_detail_true
        original_bg += list(np.diag(bg))
        optimal_bs += BS_under_AG_fine_detail_true
        original_bs += list(np.diag(bs))

    else:
        continue


In [ ]:
print('AG: ')
print(len(optimal), len(original))
print('Optimal', 'Original', 'Random')
print('mean: ', np.mean(optimal), np.mean(original), np.mean(random))
print('sd: ', np.std(optimal), np.std(original), np.std(random))

In [ ]:
###Text Only
root_dir = os.path.join(parent_dir, "In-Silico_Experiments_random/Pseudo-Magazines")

#Within Magazines
optimal = []
original = []
random = []

optimal_bg = []
original_bg = []
random_bg = []

optimal_bs = []
original_bs = []
random_bs = []

ags = []

avg_indices = None 
avg_high_level = None 
zero_indices = None 


for f in range(2):
    print('Magazine '+str(f+1))
    dir = root_dir + '/M' + str(f+1) + '/Var_inputs.pt'
    dir_text_only = root_dir + '/M' + str(f+1) + '/Var_inputs_Text_Only.pt'
    if os.path.exists(dir):
        Var_inputs = torch.load(dir)
        Var_inputs_text_only = torch.load(dir_text_only)
        ag = 0
        bg = 0
        bs = 0
        ag_avg = 0
        bg_avg = 0
        bs_avg = 0

        for k in range(10):
            xgb_model = xgb.XGBRegressor()
            xgb_model.load_model(os.path.join(parent_dir,'src/Ad_Gaze_Model/10_models/Model_'+str(k+1)+'.json'))
            
            ag += xgb_model.predict(Var_inputs[k])/10

            xgb_model.load_model(os.path.join(parent_dir,'In-Silico_Experiments_random/Text_Only_Vars/10_models_AG/Model_'+str(k+1)+'.json'))
            ag_avg += xgb_model.predict(Var_inputs_text_only[k])/10

            xgb_model.load_model(os.path.join(parent_dir,'src/Brand_Gaze_Model/10_models/Model_'+str(k+1)+'.json'))
            bg += xgb_model.predict(Var_inputs[k])/10

            xgb_model.load_model(os.path.join(parent_dir,'src/Brand_Share_Model/10_models/Model_'+str(k+1)+'.json'))
            bs += xgb_model.predict(Var_inputs[k])/10

        ag = np.exp(ag)-1
        bg = np.exp(bg)-1
        ag_avg = np.exp(ag_avg)-1 #if avg_indices is not None else ag

        N = int(np.sqrt(len(ag)))
        ag = ag.reshape((N,N))
        bg = bg.reshape((N,N))
        bs = bs.reshape((N,N))
        ag_avg = ag_avg.reshape((N,N))

        arr = np.arange(N)
        np.random.shuffle(arr)
        for i in range(N):
            random.append(ag[i,arr[i]])
            random_bg.append(bg[i,arr[i]])
            random_bs.append(bs[i,arr[i]])

        workers = []
        jobs = []
        Assign_ids = []
        for i in range(N):
            workers.append(i+1)
            jobs.append(i+1)
            Assign_ids.append(i)

        ag_assign = ag_avg

        max_ad_attention = np.max(ag_assign)
        Ad_Gaze_cost = max_ad_attention - ag_assign

        Prob_solved_Ad = Assignment_Problem(Ad_Gaze_cost, workers, jobs)

        Ad_Gaze_true = ag
        Brand_Gaze_true = bg
        Brand_Share_true = bs

        strategy_AG = ''
        BG_under_AG_assignment = 0
        BS_under_AG_assignment = 0
        AG_fine_detail = []
        BG_under_AG_fine_detail = []
        BS_under_AG_fine_detail = []
        AG_fine_detail_true = []
        BG_under_AG_fine_detail_true = []
        BS_under_AG_fine_detail_true = []
        for v in Prob_solved_Ad.variables():
            if v.varValue == 1:
                curr = (v.name).split('_')
                temp = curr[0]+' Ad '+str(Assign_ids[int(curr[1])-1])+' to Counterpage '+str(Assign_ids[int(curr[2])-1])
                AG_fine_detail.append(max_ad_attention-Ad_Gaze_cost[int(curr[1])-1,int(curr[2])-1])
                AG_fine_detail_true.append(Ad_Gaze_true[int(curr[1])-1,int(curr[2])-1])

                # BG_under_AG_assignment += Brand_Gaze_cost[int(curr[1])-1,int(curr[2])-1]
                # BG_under_AG_fine_detail.append(max_brand_attention-Brand_Gaze_cost[int(curr[1])-1,int(curr[2])-1])
                BG_under_AG_fine_detail_true.append(Brand_Gaze_true[int(curr[1])-1,int(curr[2])-1])

                # BS_under_AG_assignment += Brand_Share_cost[int(curr[1])-1,int(curr[2])-1]
                # BS_under_AG_fine_detail.append(max_brand_share-Brand_Share_cost[int(curr[1])-1,int(curr[2])-1])
                BS_under_AG_fine_detail_true.append(Brand_Share_true[int(curr[1])-1,int(curr[2])-1])
                strategy_AG += temp+'; '
                # print(temp)
        
        optimal += AG_fine_detail_true
        original += list(np.diag(ag))
        optimal_bg += BG_under_AG_fine_detail_true
        original_bg += list(np.diag(bg))
        optimal_bs += BS_under_AG_fine_detail_true
        original_bs += list(np.diag(bs))

    else:
        continue


In [ ]:
print('AG: ')
print(len(optimal), len(original))
print('Text-Based-Optimal', 'Original', 'Random')
print('mean: ', np.mean(optimal), np.mean(original), np.mean(random))
print('sd: ', np.std(optimal), np.std(original), np.std(random))

### Across Magazines

In [ ]:
root_dir = os.path.join(parent_dir, "In-Silico_Experiments_random/Pseudo-Magazines")

size_tot = 0
ag = 0
bg = 0
bs = 0
ag_avg = 0
bg_avg = 0
bs_avg = 0

avg_indices = None 
avg_high_level = None 
zero_indices = None 

#To run Observed-features-only experiment, uncomment the following lines:
# avg_indices = np.array(list(range(4,56))+list(range(67,71))+[109])
# avg_high_level = torch.load(os.path.join(parent_dir, 'In-Silico_Experiments/average_nonobserv.pt')) 
# zero_indices = np.array([110])

#To run Low-Level-features-only experiment, uncomment the following lines:
# avg_indices = np.array([20,21,22,23,50,51,52,53,54,55,109])
# avg_high_level = torch.load(os.path.join(parent_dir, 'In-Silico_Experiments/average_high_level.pt'))
# zero_indices = np.array(list(range(56,65))+list(range(67,109))+[110])

for k in range(10):
    cat_vars = []
    xgb_model = xgb.XGBRegressor()
    
    for i in range(9):
        try:
            curr_root = root_dir + '/Total/Var_inputs_portion'+str(i)+'.pt'
            curr_vars = torch.load(curr_root)
            cat_vars.append(curr_vars[k])
        except:
            continue
    
    cat_vars = np.concatenate(cat_vars, axis=0)

    if avg_indices is not None:
        cat_vars_avg = cat_vars.copy()
        cat_vars_avg[:, avg_indices] = avg_high_level
        cat_vars_avg[:, zero_indices] = 0

    xgb_model.load_model(os.path.join(parent_dir,'src/Ad_Gaze_Model/10_models/Model_'+str(k+1)+'.json'))
    ag += xgb_model.predict(cat_vars)/10
    if avg_indices is not None:
        ag_avg += xgb_model.predict(cat_vars_avg)/10


    xgb_model.load_model(os.path.join(parent_dir,'src/Brand_Gaze_Model/10_models/Model_'+str(k+1)+'.json'))
    bg += xgb_model.predict(cat_vars)/10
    if avg_indices is not None:
        bg_avg += xgb_model.predict(cat_vars_avg)/10

    xgb_model.load_model(os.path.join(parent_dir,'src/Brand_Share_Model/10_models/Model_'+str(k+1)+'.json'))
    bs += xgb_model.predict(cat_vars)/10
    if avg_indices is not None:
        bs_avg += xgb_model.predict(cat_vars_avg)/10
    

In [ ]:
original_across = []
optimal_across = []
original_across_bg = []
optimal_across_bg = []
original_across_bs = [] 
optimal_across_bs = []

ag = np.exp(ag)-1
bg = np.exp(bg)-1

if avg_indices is not None:
    ag_avg = np.exp(ag_avg)-1 
    bg_avg = np.exp(bg_avg)-1
    bs_avg = np.exp(bs_avg)-1
else:
    ag_avg = np.zeros_like(ag) 
    bg_avg = np.zeros_like(bg)
    bs_avg = np.zeros_like(bs)

N = int(np.sqrt(len(ag)))
ag = ag.reshape((N,N))
bg = bg.reshape((N,N))
bs = bs.reshape((N,N))
ag_avg = ag_avg.reshape((N,N))
bg_avg = bg_avg.reshape((N,N))
bs_avg = bs_avg.reshape((N,N))

workers = []
jobs = []
Assign_ids = []
for i in range(N):
    workers.append(i+1)
    jobs.append(i+1)
    Assign_ids.append(i)

ag_assign = ag_avg if avg_indices is not None else ag
bg_assign = bg_avg if avg_indices is not None else bg
bs_assign = bs_avg if avg_indices is not None else bs

max_ad_attention = np.max(ag_assign)
max_brand_attention = np.max(bg_assign)
max_brand_share = np.max(bs_assign)
Ad_Gaze_cost = max_ad_attention - ag_assign
Brand_Gaze_cost = max_brand_attention - bg_assign
Brand_Share_cost = max_brand_share - bs_assign

Prob_solved_Ad = Assignment_Problem(Ad_Gaze_cost, workers, jobs)

Ad_Gaze_true = ag
Brand_Gaze_true = bg
Brand_Share_true = bs

strategy_AG = ''
BG_under_AG_assignment = 0
BS_under_AG_assignment = 0
AG_fine_detail = []
BG_under_AG_fine_detail = []
BS_under_AG_fine_detail = []
AG_fine_detail_true = []
BG_under_AG_fine_detail_true = []
BS_under_AG_fine_detail_true = []
for v in Prob_solved_Ad.variables():
    if v.varValue == 1:
        curr = (v.name).split('_')
        temp = curr[0]+' Ad '+str(Assign_ids[int(curr[1])-1])+' to Counterpage '+str(Assign_ids[int(curr[2])-1])
        AG_fine_detail.append(max_ad_attention-Ad_Gaze_cost[int(curr[1])-1,int(curr[2])-1])
        AG_fine_detail_true.append(Ad_Gaze_true[int(curr[1])-1,int(curr[2])-1])

        BG_under_AG_assignment += Brand_Gaze_cost[int(curr[1])-1,int(curr[2])-1]
        BG_under_AG_fine_detail.append(max_brand_attention-Brand_Gaze_cost[int(curr[1])-1,int(curr[2])-1])
        BG_under_AG_fine_detail_true.append(Brand_Gaze_true[int(curr[1])-1,int(curr[2])-1])

        BS_under_AG_assignment += Brand_Share_cost[int(curr[1])-1,int(curr[2])-1]
        BS_under_AG_fine_detail.append(max_brand_share-Brand_Share_cost[int(curr[1])-1,int(curr[2])-1])
        BS_under_AG_fine_detail_true.append(Brand_Share_true[int(curr[1])-1,int(curr[2])-1])
        strategy_AG += temp+'; '
        # print(temp)

optimal_across += AG_fine_detail_true
original_across += list(np.diag(ag))
optimal_across_bg += BG_under_AG_fine_detail_true
original_across_bg += list(np.diag(bg))
optimal_across_bs += BS_under_AG_fine_detail_true
original_across_bs += list(np.diag(bs))

In [ ]:
print('AG: ')
print('Optimal', 'Original')
print(np.mean(optimal_across), np.mean(original_across))
print(np.std(optimal_across), np.std(original_across))

In [ ]:
###Text Only
size_tot = 0
ag = 0
bg = 0
bs = 0
ag_avg = 0
bg_avg = 0
bs_avg = 0

for k in range(10):
    cat_vars = []
    cat_vars_text_only = []
    xgb_model = xgb.XGBRegressor()
    
    for i in range(9):
        try:
            curr_root = root_dir + '/Total/Var_inputs_portion'+str(i)+'.pt'
            curr_root_text_only = root_dir + '/Total/Var_inputs_portion'+str(i)+'_Text_Only.pt'
            curr_vars = torch.load(curr_root)
            curr_vars_text_only = torch.load(curr_root_text_only)
            cat_vars.append(curr_vars[k])
            cat_vars_text_only.append(curr_vars_text_only[k])
        except:
            continue
    
    cat_vars = np.concatenate(cat_vars, axis=0)
    cat_vars_text_only = np.concatenate(cat_vars_text_only, axis=0)

    xgb_model.load_model(os.path.join(parent_dir,'src/Ad_Gaze_Model/10_models/Model_'+str(k+1)+'.json'))
    ag += xgb_model.predict(cat_vars)/10
    xgb_model.load_model(os.path.join(parent_dir,'In-Silico_Experiments_random/Text_Only_Vars/10_models_AG/Model_'+str(k+1)+'.json'))
    ag_avg += xgb_model.predict(cat_vars_text_only)/10


    xgb_model.load_model(os.path.join(parent_dir,'src/Brand_Gaze_Model/10_models/Model_'+str(k+1)+'.json'))
    bg += xgb_model.predict(cat_vars)/10

    xgb_model.load_model(os.path.join(parent_dir,'src/Brand_Share_Model/10_models/Model_'+str(k+1)+'.json'))
    bs += xgb_model.predict(cat_vars)/10
    
original_across = []
optimal_across = []
original_across_bg = []
optimal_across_bg = []
original_across_bs = [] 
optimal_across_bs = []

ag = np.exp(ag)-1
bg = np.exp(bg)-1
ag_avg = np.exp(ag_avg)-1 #if avg_indices is not None else ag

N = int(np.sqrt(len(ag)))
ag = ag.reshape((N,N))
bg = bg.reshape((N,N))
bs = bs.reshape((N,N))
ag_avg = ag_avg.reshape((N,N))

workers = []
jobs = []
Assign_ids = []
for i in range(N):
    workers.append(i+1)
    jobs.append(i+1)
    Assign_ids.append(i)

ag_assign = ag_avg

max_ad_attention = np.max(ag_assign)
Ad_Gaze_cost = max_ad_attention - ag_assign

Prob_solved_Ad = Assignment_Problem(Ad_Gaze_cost, workers, jobs)

Ad_Gaze_true = ag
Brand_Gaze_true = bg
Brand_Share_true = bs

strategy_AG = ''
BG_under_AG_assignment = 0
BS_under_AG_assignment = 0
AG_fine_detail = []
BG_under_AG_fine_detail = []
BS_under_AG_fine_detail = []
AG_fine_detail_true = []
BG_under_AG_fine_detail_true = []
BS_under_AG_fine_detail_true = []
for v in Prob_solved_Ad.variables():
    if v.varValue == 1:
        curr = (v.name).split('_')
        temp = curr[0]+' Ad '+str(Assign_ids[int(curr[1])-1])+' to Counterpage '+str(Assign_ids[int(curr[2])-1])
        AG_fine_detail.append(max_ad_attention-Ad_Gaze_cost[int(curr[1])-1,int(curr[2])-1])
        AG_fine_detail_true.append(Ad_Gaze_true[int(curr[1])-1,int(curr[2])-1])

        # BG_under_AG_assignment += Brand_Gaze_cost[int(curr[1])-1,int(curr[2])-1]
        # BG_under_AG_fine_detail.append(max_brand_attention-Brand_Gaze_cost[int(curr[1])-1,int(curr[2])-1])
        BG_under_AG_fine_detail_true.append(Brand_Gaze_true[int(curr[1])-1,int(curr[2])-1])

        # BS_under_AG_assignment += Brand_Share_cost[int(curr[1])-1,int(curr[2])-1]
        # BS_under_AG_fine_detail.append(max_brand_share-Brand_Share_cost[int(curr[1])-1,int(curr[2])-1])
        BS_under_AG_fine_detail_true.append(Brand_Share_true[int(curr[1])-1,int(curr[2])-1])
        strategy_AG += temp+'; '
        # print(temp)

optimal_across += AG_fine_detail_true
original_across += list(np.diag(ag))
optimal_across_bg += BG_under_AG_fine_detail_true
original_across_bg += list(np.diag(bg))
optimal_across_bs += BS_under_AG_fine_detail_true
original_across_bs += list(np.diag(bs))

In [ ]:
print('AG: ')
print('Text-Based-Optimal', 'Original')
print(len(optimal_across), len(original_across))
print(np.mean(optimal_across), np.mean(original_across))
print(np.std(optimal_across), np.std(original_across))

In [ ]:
#Across-Magazines Random Assignment
import random

random.seed(0)

arr = np.arange(50)

random.shuffle(arr)

optimal_ag_random = []
optimal_bg_random = []
optimal_bs_random = []

for i in range(len(arr)):
    optimal_ag_random.append(ag[i,arr[i]])
    optimal_bg_random.append(bg[i,arr[i]])
    optimal_bs_random.append(bs[i,arr[i]])

print('AG Random: ')
print(np.mean(optimal_ag_random), np.std(optimal_ag_random))